<div class="alert alert-block alert-info">
                                            Regular Expressions
    <p><a href= "https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html">Pandas documentation for text</a>
    <p><a href = 'https://www.analyticsvidhya.com/blog/2014/11/text-data-cleaning-steps-python/'> Analytics Vidhya Data Cleaning</a> 
    <p><a href = 'http://ieva.rocks/2016/08/07/cleaning-text-for-nlp/'> NLP Data Cleaning</a> 
        
</div>

![alt Regexp](FIG1.png "Title")

![alt Regexp](FIG2.png "Title")


<div class='alert alert-block alert-info'>
<li>[^] ( within [ ] ) is negation in regular expression whereas ^ is "begining of string"</li>

<li>[^a-z] matches any single character that is not from "a" to "z"</li>

<li>^[a-z] means string starts with from "a" to "z"</li>

</div>

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [12]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.iloc[0]

'03/25/93 Total time of visit (in minutes):\n'

In [20]:
pd.set_option('max_colwidth', 200)
pd.set_option('max_rows', 500)



In [14]:
df.shape
df.iloc[0:7]

0                        03/25/93 Total time of visit (in minutes):\n
1                                      6/18/85 Primary Care Doctor:\n
2            sshe plans to move as of 7/8/71 In-Home Services: None\n
3                               7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
5                                   .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current substance abuse:\n
dtype: object

<div class='alert alert-block alert-info'> 
The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.
</div>

In [17]:
df = pd.DataFrame(df, columns=['Text'])

In [23]:
df.iloc[0:100]

,Text,text_contains_month
0,03/25/93 Total time of visit (in minutes):\n,False
1,6/18/85 Primary Care Doctor:\n,False
2,sshe plans to move as of 7/8/71 In-Home Services: None\n,False
3,7 on 9/27/75 Audit C Score Current:\n,False
4,2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n,False
5,.Per 7/06/79 Movement D/O note:\n,False
6,"4, 5/18/78 Patient's thoughts about current substance abuse:\n",False
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n,False
8,3/7/86 SOS-10 Total Score:\n,False
9,(4/10/71)Score-1Audit C Score Current:\n,False


In [22]:
df = df.assign(text_contains_month = df['Text'].str.contains('Mar'))

In [24]:
df[df['text_contains_month']]

,Text,text_contains_month
108,"PET Scan (DPSH 5/04/74): 1) Marked hypometabolism involving the bilateral caudate nuclei. This is a non-specific finding, although this raises the question of a multisystem atrophy, specifically M...",True
127,see above and APS eval of 26 May 1982 Social History Marital Status: Single\n,True
150,12 March 1980 SOS-10 Total Score:\n,True
154,06 Mar 1974 Primary Care Doctor:\n,True
162,dON 05 Mar 1974\n,True
185,see 17 Aug 1985 eval by Dr. Ngo.Social History Marital Status: Married\n,True
188,12 March 2004 CPT Code: 90801 - Psychiatric Diagnosis Interview\n,True
204,"Mar. 10, 1976 CPT Code: 90791: No medical services\n",True
287,.The patient reports onset of depressed mood SINCE ~6 MONTHS s/p liver transplant in Mar 2010 associated with:\n,True
289,NV fire fighter died Sep 2007 while working. Was friend from deployment to San Marino and trainings for years prior. Still troubling to pt. Didn't go to his funeral. Spiritual/Religion:\n,True


### List of variations

<div class='alert alert-block alert-info'>
<li> 04/20/2009; 04/20/09; 4/20/09; 4/3/09</li>
<li> Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;</li>
<li> 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009</li>
<li> Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009</li>
<li> Feb 2009; Sep 2009; Oct 2010</li>
<li> 6/2008; 12/2009</li>
<li> 2009; 2010</li>
</div>    

<div class='alert alert-block alert-success'>
Check for dates with all three components
    <li> 04/20/2009; 04/20/09; 4/20/09; 4/3/09</li>
</div>


In [81]:
df_date_extracted = df['Text'].str.extractall(r'(?P<FullDate>(?P<Month>\d?\d)/(?P<Day>\d?\d)/(?P<Year>\d?\d?\d\d))')



#Check for four digit dates
#df_date_extracted[df_date_extracted['Year'].str.len() == 4]

#Check for two digit dates
#df_date_extracted[df_date_extracted['Year'].str.len() == 2]
#check for year equal to three digits
df_date_extracted[(df_date_extracted['Year'].str.len() == 3)]
#72	1	9/36/308	9	36	

# This is the CBC count and so drop these
df[df['Text'].str.contains('9/36/308')]

#Remove  if dates year is NOT 2 digits or is not 4 digits
df_date_extracted_daymonth_year = df_date_extracted[(df_date_extracted['Year'].str.len() == 2) | (df_date_extracted['Year'].str.len() == 4)]



In [103]:
df_date_extracted_daymonth_year

,,FullDate,Month,Day,Year
,match,,,,
0,0,03/25/93,03,25,93
1,0,6/18/85,6,18,85
2,0,7/8/71,7,8,71
3,0,9/27/75,9,27,75
4,0,2/6/96,2,6,96
5,0,7/06/79,7,06,79
6,0,5/18/78,5,18,78
7,0,10/24/89,10,24,89
8,0,3/7/86,3,7,86


<div class='alert alert-block alert-success'>
Check for dates with only two of the three components
<li>6/2008; 12/2009 </li>
<li> Does not start with one or two digits, but has month with one or two digits and years with four digits </li>    
</div>

In [123]:
df_date_extracted_month_year = df['Text'].str.extractall(r'(?P<FullDate>(?P<Month>^[\d?\d]\d?\d)/(?P<Year>\d\d\d\d)\b)')


In [124]:
df_date_extracted_month_year['Day'] = '01'

In [125]:
df_date_extracted_month_year

,,FullDate,Month,Year,Day
,match,,,,
345,0,10/1973,10,1973,01
348,0,12/2005,12,2005,01
353,0,10/1997,10,1997,01
360,0,12/2008,12,2008,01
363,0,12/1975,12,1975,01
375,0,06/1973,06,1973,01
390,0,08/1988,08,1988,01
404,0,10/1986,10,1986,01
406,0,12/1994,12,1994,01


<div class='alert alert-block alert-success'>
Check for dates with only year
<li>2008; 2009 </li>
<li> Word boundary followed by year </li>    
</div>

In [143]:
df_date_extracted_just_year = df['Text'].str.extractall(r'[ ]+(?P<Year>\d{4,4})\b')

In [146]:
df_date_extracted_just_year = df_date_extracted_just_year.assign(Day=1, Month=1) # 251
#df[df['Text'].str.contains('\\b'9079', regex=True)



In [147]:
df_date_extracted_just_year

,,Year,Day,Month
,match,,,
125,0,2001,1,1
126,0,2004,1,1
127,0,1982,1,1
128,0,2002,1,1
129,0,1972,1,1
130,0,1987,1,1
131,0,1996,1,1
132,0,2007,1,1
133,0,1994,1,1


In [41]:
# from dateutil.parser import parse
# from dateparser.search import search_dates
# # parse("Today is December 1, 2018", fuzzy_with_tokens=True)
 
# # parse("Today is Dec 1 2018", fuzzy_with_tokens=True)
 
# # parse("Today is Dec 1", fuzzy_with_tokens=True)

# def searchForDate(x):
# #     return parse(x, fuzzy_with_tokens=True)
#     return search_dates(x)


# df['Text'].apply(searchForDate)


In [ ]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here

In [ ]:
import pandas as pd

time_sentences = ["Monday: The doctor's appointment is at 2:45pm.", 
                  "Tuesday: The dentist's appointment is at 11:30 am.",
                  "Wednesday: At 7:00pm, there is a basketball game!",
                  "Thursday: Be back home by 11:15 pm at the latest.",
                  "Friday: Take the train at 08:10 am, arrive at 09:00am."]

df = pd.DataFrame(time_sentences, columns=['text'])
df

####  Find the number of characters for each string in df['text']

In [43]:

df = df.assign(text_length = df['text'].apply(len))

#### Find the number of tokens for each string in df['text']

In [44]:

# df.assign(split_text = df['text'].str.split())
df_expanded = df['text'].str.split(expand=True)

df_expanded


,0,1,2,3,4,5,6,7,8,9
0,Monday:,The,doctor's,appointment,is,at,2:45pm.,None,None,None
1,Tuesday:,The,dentist's,appointment,is,at,11:30,am.,None,None
2,Wednesday:,At,"7:00pm,",there,is,a,basketball,game!,None,None
3,Thursday:,Be,back,home,by,11:15,pm,at,the,latest.
4,Friday:,Take,the,train,at,08:10,"am,",arrive,at,09:00am.


####  Find which entries contain the word 'appointment'

In [47]:
df = df.assign(text_contains_appointment = df['text'].str.contains('appointment'))

####  Find how many times a digit occurs in each string

<div class="alert alert-block alert-success">
<h5>Two methods shown - One gets the count of digits and the other uses a powerful function "enumerate" to loop through the string . It allows us to loop over something and have an automatic counter.</h5>
</div>

In [50]:
df = df.assign(contains_digits = df['text'].str.count(r'\d'))

In [65]:
def indexOfDigit(text):
    return ({c: i  for i, c in enumerate(text) if c.isdigit()})
    
df['text'].apply(indexOfDigit)

0             {'2': 39, '4': 41, '5': 42}
1             {'1': 42, '3': 44, '0': 45}
2                      {'7': 14, '0': 17}
3                      {'1': 29, '5': 30}
4    {'0': 50, '8': 27, '1': 29, '9': 47}
Name: text, dtype: object

####  Find all occurances of the digits

In [67]:

df['text'].str.findall(r'\d')

0                   [2, 4, 5]
1                [1, 1, 3, 0]
2                   [7, 0, 0]
3                [1, 1, 1, 5]
4    [0, 8, 1, 0, 0, 9, 0, 0]
Name: text, dtype: object

![alt Regexp](FIG3.png "Title")



In [78]:
# group and find the hours and minutes
df['text'].str.findall(r'(\d?\d):(\d?\d)')

0               [(2, 45)]
1              [(11, 30)]
2               [(7, 00)]
3              [(11, 15)]
4    [(08, 10), (09, 00)]
Name: text, dtype: object

In [71]:
# replace weekdays with '???'
df['text'].str.replace(r'\w+day\b', '???')

0          ???: The doctor's appointment is at 2:45pm.
1       ???: The dentist's appointment is at 11:30 am.
2          ???: At 7:00pm, there is a basketball game!
3         ???: Be back home by 11:15 pm at the latest.
4    ???: Take the train at 08:10 am, arrive at 09:...
Name: text, dtype: object

In [79]:
# replace weekdays with 3 letter abbrevations
df['text'].str.replace(r'(\w+day\b)', lambda x: x.groups()[0][:3])

0          Mon: The doctor's appointment is at 2:45pm.
1       Tue: The dentist's appointment is at 11:30 am.
2          Wed: At 7:00pm, there is a basketball game!
3         Thu: Be back home by 11:15 pm at the latest.
4    Fri: Take the train at 08:10 am, arrive at 09:...
Name: text, dtype: object

In [81]:
# create new columns from first match of extracted groups
df['text'].str.extract(r'(\d?\d):(\d\d)')

,0,1
0,2,45
1,11,30
2,7,00
3,11,15
4,08,10


In [82]:
# extract the entire time, the hours, the minutes, and the period
df['text'].str.extractall(r'((\d?\d):(\d\d) ?([ap]m))')

0   1   2   3
  match                      
0 0        2:45pm   2  45  pm
1 0      11:30 am  11  30  am
2 0        7:00pm   7  00  pm
3 0      11:15 pm  11  15  pm
4 0      08:10 am  08  10  am
  1       09:00am  09  00  am

In [83]:
# extract the entire time, the hours, the minutes, and the period with group names
df['text'].str.extractall(r'(?P<time>(?P<hour>\d?\d):(?P<minute>\d\d) ?(?P<period>[ap]m))')

time hour minute period
  match                             
0 0        2:45pm    2     45     pm
1 0      11:30 am   11     30     am
2 0        7:00pm    7     00     pm
3 0      11:15 pm   11     15     pm
4 0      08:10 am   08     10     am
  1       09:00am   09     00     am